In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from itertools import chain
from sys import platform
from copy import deepcopy as dc
from scipy.interpolate import interp1d
from astropy.time import Time

vims_wave = np.loadtxt('vims_wave.txt')

new_array = ['_0405','_0607','_0809','_1011','_1213','_1415','_1617']
if platform == 'darwin' or platform == 'win32' or platform == 'linux':
    for number in new_array:
        new_string = 'df'+number+' = pd.read_pickle("total_data/data'+number+'.pkl")'
        exec(new_string)
else:
    df_list = []
    for number in new_array:
        new_string = 'df'+number+' = pd.read_csv("total_data/data'+number+'.csv")'
        exec(new_string)
        df_list.append(locals()['df'+number])
    for df in df_list:
        for i in range(len(df['spectrum'])):
            df['spectrum'][i] = np.array([float(x) for x in df['spectrum'][i][1:-1].split(',')])

vims_list = []
for number in new_array:
    exec('vims'+number+' = np.average([np.load("vims_shift/vims_20'+number[1:3]+'.npy"),np.load("vims_shift/vims_20'+number[3:5]+'.npy")],axis=0)')
    vims_list.append(locals()['vims'+number])

def powerlaw(x,a,b):
    return a*np.power(x,b)

def gaussian(x,a,mu,sigma):
    return a*np.exp(-(x-mu)**2/(2*sigma**2))

def flatten(some_list):
    flat_list = []
    for element in some_list:
        if type(element) is list or type(element) is tuple:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

band_channels = list(chain(range(29,35),range(46,60),range(78,96),range(102,106)))

window = list(range(56,83))

def fit_line(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_fit = powerlaw(vims_wave,*my_fit)
    return final_fit

def custom_fit(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_spectra = spectra-powerlaw(vims_wave,*my_fit)
    return final_spectra

def single_fit(spectrum):
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectrum[band_channels],p0=[.15,-12])
    final_spectrum = spectrum-powerlaw(vims_wave,*my_fit)
    return final_spectrum

def shift_ret(spectrum):
    my_fit,_ = curve_fit(gaussian,vims_wave[window],spectrum[window],p0=[.05,2.05])
    return my_fit[1]

In [6]:
scatter_list = []
for number in new_array:
    my_string = 'scatter'+number+" = df"+number
    exec(my_string)
    scatter_list.append(locals()['scatter'+number])

total_scatter = pd.concat(scatter_list,ignore_index=True)

all_spectra = total_scatter['spectrum']

averaged_total = np.average(all_spectra)

spec_list = []
for number in new_array:
    my_string = 'old_spec'+number+' = df'+number+"['spectrum']"
    exec(my_string)
    new_string = 'spec'+number+' = np.array([np.abs(x) for x in old_spec'+number+'])'
    exec(new_string)
    spec = 'spec'+number
    spec_list.append(locals()[spec])

averaged_list = []
for number in new_array:
    my_string = 'averaged'+number+' = np.average(spec'+number+',axis=0)'
    exec(my_string)
    averaged_list.append(locals()['averaged'+number])

copy_list = []
for number in new_array:
    my_string = 'copy_spec'+number+' = dc(spec'+number+')'
    exec(my_string)
    copy_spec = 'copy_spec'+number
    copy_list.append(locals()[copy_spec])

for (spec,copy_spec) in zip(spec_list,copy_list):
    for i in range(256):
        filtered = False
        k = 0
        while filtered == False:
            mean_1 = np.mean(copy_spec[:,i])
            median = np.median(copy_spec[:,i])
            std = np.std(copy_spec[:,i])
            vals = copy_spec[:,i]
            ind = np.where(np.abs(vals-mean_1)>2*std)
            copy_spec[ind,i] = median
            mean_2 = np.mean(copy_spec[:,i])
            k += 1
            if np.abs(mean_1-mean_2) <=.05*mean_1:
                filtered = True

pList_total = []
for (copy_spec,number) in zip(copy_list,new_array):
    exec('pList'+number+' = []')
    for spec in copy_spec:
        new_spec = single_fit(spec)
        final_spec = np.abs(new_spec)
        locals()['pList'+number].append(final_spec)
    pList_total.append(locals()['pList'+number])



In [3]:
total_scatter

,fileid,lineres,emission,incidence,latitude,longitude,starttime,spectrum
0,19779,62097.681413,30.881187,39.996733,10.408238,309.431710,2005-03-31 14:17:04.096,"[0.0627468, 0.0772501, 0.115962, 0.158278, 0.1..."
1,73581,15042.961556,30.017625,39.995293,10.798510,305.135524,2005-04-16 17:50:04.202,"[0.0649121, 0.0794369, 0.119561, 0.161287, 0.1..."
2,73581,15275.300926,33.924880,39.988389,13.097070,301.397352,2005-04-16 17:50:04.202,"[0.0686598, 0.0839468, 0.126895, 0.169983, 0.1..."
3,73581,15519.941008,34.936784,39.987750,13.601726,300.441459,2005-04-16 17:50:04.202,"[0.0683702, 0.0855701, 0.127764, 0.171474, 0.1..."
4,73581,13934.837773,31.017054,39.983581,11.435282,304.167414,2005-04-16 17:50:04.202,"[0.0649137, 0.0816108, 0.120783, 0.164939, 0.1..."
...,...,...,...,...,...,...,...,...
1270,27343,38414.199284,31.189751,20.247050,12.489629,310.872401,2016-01-31 21:20:57.961,"[0.0603998, 0.103166, 0.163553, 0.156522, 0.09..."
1271,27864,37707.637871,25.783813,20.176448,9.217981,315.052787,2016-01-31 21:17:05.804,"[0.0590715, 0.100256, 0.157004, 0.149544, 0.09..."
1272,27650,37434.752350,25.709792,20.136072,9.204018,315.129655,2016-01-31 21:18:23.204,"[0.059886, 0.100492, 0.15705, 0.1499, 0.096029..."
1273,59077,49468.523178,32.391633,20.104393,10.177082,315.082358,2016-02-16 19:06:13.322,"[0.0598412, 0.10206, 0.160232, 0.15152, 0.0967..."


In [16]:
for df in scatter_list:
    for i in range(len(df['starttime'])):
        df['starttime'][i] = Time(df['starttime'][i].replace(' ','T'),format='isot').jd

C:\Users\lucas\AppData\Local\Temp/ipykernel_16024/2045884312.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['starttime'][i] = Time(df['starttime'][i].replace(' ','T'),format='isot').jd


In [33]:
shift_dates = [
'2004-01-01T00:00:00.00',
'2005-01-01T00:00:00.00',
'2005-07-01T00:00:00.00',
'2006-01-01T00:00:00.00',
'2006-07-01T00:00:00.00',
'2007-01-01T00:00:00.00',
'2007-07-01T00:00:00.00',
'2008-01-01T00:00:00.00',
'2008-07-01T00:00:00.00',
'2009-01-01T00:00:00.00',
'2009-07-01T00:00:00.00',
'2010-01-01T00:00:00.00',
'2010-07-01T00:00:00.00',
'2011-01-01T00:00:00.00',
'2011-07-01T00:00:00.00',
'2012-01-01T00:00:00.00',
'2012-07-01T00:00:00.00',
'2013-01-01T00:00:00.00',
'2013-07-01T00:00:00.00',
'2014-01-01T00:00:00.00',
'2014-04-19T00:00:00.00',
'2014-07-01T00:00:00.00',
'2015-01-01T00:00:00.00',
'2015-07-01T00:00:00.00',
'2016-01-01T00:00:00.00',
'2016-07-01T00:00:00.00',
'2017-01-01T00:00:00.00',
'2017-07-01T00:00:00.00',
'2017-10-19T00:00:00.00',
]

shift_vals = [0.0,0.0,0.4,1.1,1.9,2.6,3.4,4.1,4.9,5.6,6.0,6.0,6.0,6.0,7.0,7.3,7.7,8.0,8.4,8.7,9.1,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8]

In [37]:
date_vector = Time(shift_dates,format='isot').jd

In [38]:
date_interp = interp1d(date_vector,shift_vals)

In [43]:
total_scatter = pd.concat(scatter_list,ignore_index=True)

In [49]:
date_interp(total_scatter['starttime'][2010])

array(0.59852618)

In [62]:
vims_2004 = np.load('vims_shift/vims_2004.npy') #! make sure to do vims_2004 instead of vims_0405 in initial cell

In [61]:
shift_2004 = vims_2004 + date_interp(total_scatter['starttime'][2010])/1000 #replicate this process for all spectra

In [60]:
shift_2004

array([0.88480854, 0.90126556, 0.91772056, 0.93417054, 0.95060855,
       0.96705055, 0.98347956, 0.9999046 , 1.0163306 , 1.0327456 ,
       1.0491606 , 1.0655526 , 1.0819576 , 1.0983605 , 1.1147635 ,
       1.1311595 , 1.1475486 , 1.1639346 , 1.1803236 , 1.1967115 ,
       1.2130976 , 1.2294825 , 1.2458656 , 1.2622435 , 1.2786236 ,
       1.2950006 , 1.3113855 , 1.3277706 , 1.3441546 , 1.3605396 ,
       1.3769145 , 1.3933026 , 1.4096955 , 1.4260895 , 1.4424865 ,
       1.4588846 , 1.4752855 , 1.4916916 , 1.5081055 , 1.5245235 ,
       1.5409485 , 1.5573776 , 1.5738095 , 1.5902495 , 1.6067156 ,
       1.6231376 , 1.6395946 , 1.6560636 , 1.6725266 , 1.6889936 ,
       1.7054656 , 1.7219396 , 1.7384166 , 1.7548956 , 1.7713746 ,
       1.7878555 , 1.8043375 , 1.8208206 , 1.8373026 , 1.8537816 ,
       1.8702606 , 1.8867335 , 1.9032085 , 1.9196775 , 1.9361455 ,
       1.9526116 , 1.9690756 , 1.9855416 , 2.0020103 , 2.0184834 ,
       2.0349605 , 2.0514424 , 2.0679333 , 2.0844324 , 2.10094